In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import tensorflow as tf
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.layers import Reshape, Flatten, Lambda
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import sigmoid
from keras.layers.merge import concatenate
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.optimizers import SGD, Adam, RMSprop
#from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')

Using TensorFlow backend.


In [15]:
import numpy as np
import glob
import cv2

In [132]:
def TreeColor(input_size1, input_size2):
    input_image = Input(shape=(input_size1, input_size2, 3))
        
    # Layer 1
    x = Conv2D(4, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input_image)
    x = BatchNormalization(name='norm_1')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = MaxPooling2D(pool_size=(4, 4))(x)

    #Layer 2
    x = Flatten()(x)
    x = Dense(16)(x)
    x = Dense(8)(x)
    x = Dense(4)(x)
    x = Dense(1)(x)

    
    model = Model(inputs = input_image, outputs=x, name='TreeColor')
    
    return model
    

In [138]:
net = TreeColor(128, 128)
net.summary()

Model: "TreeColor"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 128, 128, 3)       0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 128, 128, 4)       108       
_________________________________________________________________
norm_1 (BatchNormalization)  (None, 128, 128, 4)       16        
_________________________________________________________________
leaky_re_lu_77 (LeakyReLU)   (None, 128, 128, 4)       0         
_________________________________________________________________
max_pooling2d_77 (MaxPooling (None, 32, 32, 4)         0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_25 (Dense)             (None, 16)                65

In [115]:
def cost_func(y_true, y_pred):
    #sig_pred = tf.sigmoid(y_pred)
    #return tf.square(y_true - sig_pred)*10
    return tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true, logits=y_pred)

In [19]:
def load(path):
    list_of_files = glob.glob(path + "/True/*.png")
    x = list()
    y = list()
    for file in list_of_files:
        img = cv2.imread(file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        x.append(img)
        y.append(1)
    
    list_of_files = glob.glob(path + "/False/*.png")
    for file in list_of_files:
        img = cv2.imread(file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        x.append(img)
        y.append(0)
    X = np.array(x)
    X = X.astype(float)
    X = X / 255
    Y = np.array(y)
    
    indices = np.arange(Y.shape[0])
    np.random.shuffle(indices)
    X = X[indices]
    Y = Y[indices]
    
    return X, Y

In [11]:
cur_path = os.getcwd()

In [23]:
x_train, y_train = load(cur_path + "/CNN/Training")
x_val, y_val = load(cur_path + "/CNN/Validation")

In [24]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(1724, 128, 128, 3)
(1724,)
(146, 128, 128, 3)
(146,)


In [25]:
optimizer = Adam (lr=0.5e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
from keras.losses import mean_squared_error

In [139]:
net.compile(loss=cost_func, optimizer=optimizer)
net.fit(x_train, y_train, validation_data = (x_val, y_val), epochs=15, batch_size=8)
net.compile(loss=mean_squared_error, optimizer='sgd', metrics = ['accuracy'])
net.save(cur_path + "/" + net.name + '.hdf5')

Train on 1724 samples, validate on 146 samples
Epoch 1/15
1724/1724 [==============================] - 4s 3ms/step - loss: 0.1431 - val_loss: 0.1387
Epoch 2/15
1724/1724 [==============================] - 3s 2ms/step - loss: 0.1015 - val_loss: 0.0639
Epoch 3/15
1724/1724 [==============================] - 3s 2ms/step - loss: 0.1009 - val_loss: 0.0289
Epoch 4/15
1724/1724 [==============================] - 3s 2ms/step - loss: 0.0911 - val_loss: 0.0205
Epoch 5/15
1724/1724 [==============================] - 3s 2ms/step - loss: 0.0913 - val_loss: 0.0229
Epoch 6/15
1724/1724 [==============================] - 3s 2ms/step - loss: 0.0611 - val_loss: 0.0238
Epoch 7/15
1724/1724 [==============================] - 3s 2ms/step - loss: 0.0605 - val_loss: 0.0276
Epoch 8/15
1724/1724 [==============================] - 3s 2ms/step - loss: 0.0491 - val_loss: 0.0321
Epoch 9/15
1724/1724 [==============================] - 3s 2ms/step - loss: 0.0515 - val_loss: 0.0242
Epoch 10/15
1724/1724 [============

In [30]:
def sigmoid(x):
    return 1/(1+ np.exp(-x))

In [44]:
x_test, y_test = load(cur_path + "/CNN/Testing")

In [45]:
print(x_test.shape)
print(y_test.shape)

(185, 128, 128, 3)
(185,)


In [140]:
from keras.models import load_model
model = load_model(cur_path + "/TreeColor.hdf5")

In [149]:
y_pred = model.predict(x_test)
y_pred = sigmoid(y_pred)

TP = 0
TN = 0
FP = 0
FN = 0

size = y_pred.size
for i in range(0, size):
    if y_pred[i] > 0.9:
        if y_test[i] == 1:
            TP +=1
        else:
            FP +=1
    else:
        if y_test[i] == 1:
            FN +=1
        else:
            TN +=1

In [150]:
print(TP)
print(TN)
print(FP)
print(FN)

85
98
0
2
